In [1]:
!pip install selenium

  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
  Using cached trio-0.28.0-py3-none-any.whl (486 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


### websites to scrap:

Global Militarisation Index
https://gmi.bicc.de/ranking-table

Global safety Index

https://www.visionofhumanity.org/maps/#/

World press freedom index:

https://rsf.org/en/index


## Global Military Index

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [2]:
# Path to Chromedriver
webdriver_path = r"f:\necessary files 2\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Base URL
base_url = "https://gmi.bicc.de/ranking-table?year={year}"

# Years to scrape
years = list(range(1990, 2022)) 

# Data storage
gmi_data = []

In [4]:
# Set up WebDriver
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

In [10]:
# Set up WebDriver
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
try:
    for year in years:
        # Navigate to the URL for the specific year
        driver.get(base_url.format(year=year))
        time.sleep(3)  # Allow time for the page to load
        
        # Wait for the table to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'table-data'))
        )

        # Locate the table
        table = driver.find_element(By.CLASS_NAME, 'table-data')
        rows = table.find_elements(By.TAG_NAME, 'tr')[1:]  # Exclude the header row

        # Extract data
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            country_data = {
                "Year": year,
                "Country": cells[0].text,
                "Military Expenditure": cells[1].text,
                "Military Personnel": cells[2].text,
                "Heavy Weapons": cells[3].text,
                "GMI Score": cells[4].text,
                "GMI Rank": cells[5].text,
                "Score Trend": cells[6].text if len(cells) > 6 else "N/A",
            }
            gmi_data.append(country_data)
finally:
    # Close the driver
    driver.quit()

In [11]:
df = pd.DataFrame(gmi_data)

In [14]:
df.sample(10)

,Year,Country,Military Expenditure,Military Personnel,Heavy Weapons,GMI Score,GMI Rank,Score Trend
3001,2010,El Salvador,0.87,0.63,0.68,79,109,→\n2.5
637,1994,Cape Verde,0.76,0.14,0.85,79,116,→\n-9.36
126,1991,Qatar,3.9,0.81,2.77,306,6,-
3250,2012,Romania,1,0.75,1.85,132,52,→\n4.86
522,1994,Israel,3.23,2.8,3.85,365,1,→\n-12.67
4044,2017,India,1.84,0.12,0.42,96,81,→\n3.75
2053,2004,Lithuania,0.99,0.76,1.34,113,78,→\n9.45
4423,2020,Singapore,1.73,1.26,2.72,210,8,→\n-19.6
1960,2003,Belize,0.64,0.67,0,47,132,→\n-0.65
3691,2015,Egypt,1.28,1.11,1.74,152,34,→\n10.22


In [15]:
df.to_csv('raw_gmi_data.csv', index=False)

## Global safety Index

In [9]:
gpi_url = "https://www.visionofhumanity.org/maps/"
# Years to scrape
years = list(range(2008, 2024))  # Adjust as needed

# Storage for data
gpi_data = []

**First I have kept the page in 2024 and check if I could extract the data of 2024 only**

In [8]:
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
driver.get(gpi_url)

# Wait for the page to load completely
time.sleep(10)  # Adjust based on your internet speed and website load time

# Locate the table element
table = driver.find_element(By.CLASS_NAME, "fHbYEu")

# Extract table rows
rows = table.find_elements(By.CSS_SELECTOR, "tbody.a tr")

# Initialize data storage
data = []

# Loop through the rows and extract information
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    rank = cols[0].text.strip()
    country = cols[1].text.strip()
    score = cols[2].text.strip()
    data.append({"Rank": rank, "Country": country, "Score": score})

# Close the driver
driver.quit()

# Create a DataFrame
gpi_df = pd.DataFrame(data)

In [10]:
gpi_df.head(10)

,Rank,Country,Score
0,1,Iceland,1.112
1,2,Ireland,1.303
2,3,Austria,1.313
3,4,New Zealand,1.323
4,5,Singapore,1.339
5,6,Switzerland,1.35
6,7,Portugal,1.372
7,8,Denmark,1.382
8,9,Slovenia,1.395
9,10,Malaysia,1.427


In [11]:
gpi_df_2024=gpi_df.copy()

**Then I have tried to change the year and see if I could change the page**

In [12]:
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

# Open the target URL
gpi_url = "https://www.visionofhumanity.org/maps/#/"
driver.get(gpi_url)

# Wait for the page to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "voh-selector"))
)

# Click on the "Year" dropdown
year_dropdown = driver.find_element(By.CLASS_NAME, "voh-selector__three")
year_dropdown.click()

# Wait for the dropdown options to appear
time.sleep(2)  # Adjust this if needed based on your network speed

# Get all year options
year_options = year_dropdown.find_elements(By.CLASS_NAME, "fzsyuK")

# Iterate through each year and click to select
for year in year_options:
    print(f"Selecting year: {year.text}")
    year.click()
    time.sleep(3)  # Allow the page to reload
    # Reopen the dropdown to select the next year
    year_dropdown = driver.find_element(By.CLASS_NAME, "voh-selector__three")
    year_dropdown.click()
    time.sleep(2)

# Close the driver
driver.quit()

Selecting year: 2023
Selecting year: 2022
Selecting year: 2021
Selecting year: 2020
Selecting year: 2019
Selecting year: 2018
Selecting year: 2017
Selecting year: 2016
Selecting year: 2015
Selecting year: 2014
Selecting year: 2013
Selecting year: 2012
Selecting year: 2011
Selecting year: 2010
Selecting year: 2009
Selecting year: 2008


**Now I can combine my findings and add them together to get the full list from 2008 to 2024**

In [10]:
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
driver.get(gpi_url)

# Wait for the page to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "voh-selector"))
)

# Initialize data storage
all_data = []

# Start iterating through the years
for year in range(2024, 2007, -1):  # Loop from 2024 to 2008
    print(f"Extracting data for year: {year}")
    
    # Click on the "Year" dropdown
    year_dropdown = driver.find_element(By.CLASS_NAME, "voh-selector__three")
    driver.execute_script("arguments[0].click();", year_dropdown)  # Use JavaScript click
    time.sleep(2)
    
    # Select the desired year
    year_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//div[contains(text(), '{year}')]"))
    )
    driver.execute_script("arguments[0].click();", year_option)  # Use JavaScript click
    time.sleep(5)  # Allow the page to reload
    
    # Locate the table element
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "fHbYEu"))
    )
    rows = table.find_elements(By.CSS_SELECTOR, "tbody.a tr")
    
    # Extract data from the table
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        rank = cols[0].text.strip()
        country = cols[1].text.strip()
        score = cols[2].text.strip()
        all_data.append({"Year": year, "Rank": rank, "Country": country, "GPI": score})

# Close the driver
driver.quit()

# Create a DataFrame
gpi_df = pd.DataFrame(all_data)

Extracting data for year: 2024
Extracting data for year: 2023
Extracting data for year: 2022
Extracting data for year: 2021
Extracting data for year: 2020
Extracting data for year: 2019
Extracting data for year: 2018
Extracting data for year: 2017
Extracting data for year: 2016
Extracting data for year: 2015
Extracting data for year: 2014
Extracting data for year: 2013
Extracting data for year: 2012
Extracting data for year: 2011
Extracting data for year: 2010
Extracting data for year: 2009
Extracting data for year: 2008


In [20]:
gpi_df[gpi_df['Year']==2008]

,Year,Rank,Country,GPI
2598,2008,1,Iceland,1.106
2599,2008,2,Denmark,1.286
2600,2008,3,Austria,1.344
2601,2008,4,New Zealand,1.353
2602,2008,5,Slovenia,1.368
...,...,...,...,...
2754,2008,157,Sudan,3.054
2755,2008,158,Russia,3.151
2756,2008,159,Somalia,3.175
2757,2008,160,Afghanistan,3.218


In [21]:
gpi_df.to_csv('raw_gpi_data.csv',index=False)

In [22]:
gpi_df_all_year=gpi_df.copy()

## Global Press Freedom

**First I have kept the page in 2024 and check if I could extract the data of 2024 only**

In [89]:
# Set up WebDriver
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

gpf_url = "https://rsf.org/en/index?year=2010"
driver.get(gpf_url)

# Wait for the page to load
time.sleep(10)

# Scraping the table data
countries = []
scores = []
ranks = []

# Find all country list items for 2024
country_elements = driver.find_elements(By.CSS_SELECTOR, ".country-list-item")

# Iterate over each country entry to extract data
for country_element in country_elements:
    name = country_element.find_element(By.CSS_SELECTOR, ".country-list-item__name").text
    score = country_element.find_element(By.CSS_SELECTOR, ".country-list-item__score").text
    rank = country_element.find_element(By.CSS_SELECTOR, ".country-list-item__position").text
    
    countries.append(name)
    scores.append(score)
    ranks.append(rank)

# Close the driver after scraping
driver.quit()

# Create a DataFrame
data = {
    'Country': countries,
    'Rank': ranks,
    'Global Score': scores
}

gpf_df = pd.DataFrame(data)

In [ ]:
gpfi_df_2024=gpf_df.copy()

In [95]:
gpfi_df_2024.head(50)

,Country,Rank,Global Score
0,Norway,1,91.89
1,Denmark,2,89.6
2,Sweden,3,88.32
3,Netherlands,4,87.73
4,Finland,5,86.55
5,Estonia,6,86.44
6,Portugal,7,85.9
7,Ireland,8,85.59
8,Switzerland,9,84.01
9,Germany,10,83.84


**Then I have tried to change the year and see if I could change the page**

In [27]:
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
    
try:
    # Navigate to the website
    driver.get("https://rsf.org/en/index")

    # Wait for the initial dropdown to be present
    wait = WebDriverWait(driver, 10)
    dropdown = wait.until(EC.presence_of_element_located((By.ID, "edit-year")))

    # Get all available years first
    select = Select(dropdown)
    years = [option.get_attribute("value") for option in select.options]

    # Iterate through each year
    for year in years:
        try:
            # Wait for the dropdown to be present and get a fresh reference
            dropdown = wait.until(EC.presence_of_element_located((By.ID, "edit-year")))
            select = Select(dropdown)

            # Select the year
            select.select_by_value(year)

            # Wait for the page to load
            time.sleep(3)

            print(f"Selected year: {year}")

        except Exception as e:
            print(f"Error selecting year {year}: {str(e)}")
            continue

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    # Close the browser
    driver.quit()

Selected year: 2024
Selected year: 2023
Selected year: 2022
Selected year: 2021
Selected year: 2020
Selected year: 2019
Selected year: 2018
Selected year: 2017
Selected year: 2016
Selected year: 2015
Selected year: 2014
Selected year: 2013
Selected year: 2012
Selected year: 2010
Selected year: 2009
Selected year: 2008
Selected year: 2007
Selected year: 2006
Selected year: 2005
Selected year: 2004
Selected year: 2003
Selected year: 2002


**Here I have iterated to 2013 only. Because website after 2013 follow different element and also a different metrics ** 

In [84]:
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

# Initialize lists to store data
all_years = []
all_countries = []
all_ranks = []
all_scores = []

try:
    driver.get("https://rsf.org/en/index")
    wait = WebDriverWait(driver, 20)
        
    # Get year dropdown
    dropdown = wait.until(EC.element_to_be_clickable((By.ID, "edit-year")))
    select = Select(dropdown)
#     years = [option.get_attribute("value") for option in select.options if option.get_attribute("value").isdigit()]
    years = [option.get_attribute("value") for option in select.options if option.get_attribute("value").isdigit() and 2013 <= int(option.get_attribute("value")) <= 2024]

    
    
    for year in years:
        try:
            # Re-find dropdown elements each iteration
            dropdown = wait.until(EC.element_to_be_clickable((By.ID, "edit-year")))
            select = Select(dropdown)
            
            # Scroll to dropdown to ensure visibility
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", dropdown)
            
            # Select the year
            select.select_by_value(year)
            print(f"Selected year: {year}")
            
            # Wait for data to load
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".loading-indicator")))  # If exists
            time.sleep(5)  # Small buffer
            
            # Wait for at least 150 country items to load
            wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, ".country-list-item")) > 150)
            
            # Scrape data
            country_elements = driver.find_elements(By.CSS_SELECTOR, ".country-list-item")
            
            for element in country_elements:
                try:
                    name = element.find_element(By.CSS_SELECTOR, ".country-list-item__name").text
                    score = element.find_element(By.CSS_SELECTOR, ".country-list-item__score").text
                    rank = element.find_element(By.CSS_SELECTOR, ".country-list-item__position").text
                    
                    all_countries.append(name)
                    all_ranks.append(rank)
                    all_scores.append(score)
                    all_years.append(year)
                except Exception as e:
                    print(f"Error scraping element: {str(e)}")
                    continue
                
            print(f"Successfully scraped data for year {year}")
            
        except Exception as e:
            print(f"Error processing year {year}: {str(e)}")
            continue

except Exception as e:
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

# Create DataFrame
gpfi_df = pd.DataFrame({
    'Year': all_years,
    'Country': all_countries,
    'Rank': all_ranks,
    'Global Score': all_scores
})

print(gpfi_df.head())

Selected year: 2024
Successfully scraped data for year 2024
Selected year: 2023
Successfully scraped data for year 2023
Selected year: 2022
Successfully scraped data for year 2022
Selected year: 2021
Successfully scraped data for year 2021
Selected year: 2020
Successfully scraped data for year 2020
Selected year: 2019
Successfully scraped data for year 2019
Selected year: 2018
Successfully scraped data for year 2018
Selected year: 2017
Successfully scraped data for year 2017
Selected year: 2016
Successfully scraped data for year 2016
Selected year: 2015
Successfully scraped data for year 2015
Selected year: 2014
Successfully scraped data for year 2014
Selected year: 2013
Successfully scraped data for year 2013
   Year      Country Rank Global Score
0  2024       Norway    1        91.89
1  2024      Denmark    2         89.6
2  2024       Sweden    3        88.32
3  2024  Netherlands    4        87.73
4  2024      Finland    5        86.55


In [87]:
gpfi_df.sample(50)

,Year,Country,Rank,Global Score
1824,2014,Cyprus,25,85.55
505,2022,Sri Lanka,146,42.13
1404,2017,South Sudan,145,51.84
1784,2015,Sri Lanka,165,39.72
1850,2014,Malta,51,76.16
1646,2015,Liechtenstein,27,82.33
557,2021,Uruguay,18,83.62
781,2020,Poland,62,71.35
554,2021,Estonia,15,84.75
244,2023,Botswana,65,64.61


In [90]:
gpfi_df_copy=gpfi_df.copy()

In [101]:
gpfi_df_copy.sample(50)

,Year,Country,Rank,Global Score
984,2019,Peru,85,69.78
1956,2014,Belarus,157,52.18
1406,2017,Mexico,147,51.03
1410,2017,Singapore,151,48.9
523,2022,Djibouti,164,35.75
1827,2014,Australia,28,83.09
1293,2017,Trinidad and Tobago,34,79.38
1659,2015,Samoa,40,77.68
1873,2014,Lesotho,74,71.64
1908,2014,Central African Republic,109,66.87


In [102]:
gpfi_df_copy.to_csv('raw_gpfi_data.csv',index=False)